In [2]:
from pytorch_lightning import Trainer, seed_everything

seed_everything(42)

from pytorch_lightning import LightningModule, LightningDataModule
import torch
from pytorch_lightning import trainer, LightningModule
from torch.nn import functional as F
import torch
import torchmetrics
import timm

from torch import nn
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings

warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

Seed set to 42


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from torch.utils.data import (
    DataLoader,
    SequentialSampler,
    RandomSampler,
    WeightedRandomSampler,
    Dataset,
)
import numpy as np


class TableDataset(Dataset):
    def __init__(self, df, features: list, label: list, num_classes=2, y_type="bt"):
        super(Dataset, self).__init__()
        assert isinstance(df, pd.DataFrame)
        assert isinstance(features, list)
        assert isinstance(label, list)

        for feature in features + label:
            assert feature in df.columns

        self.df = df.dropna(subset=features + label)
        assert len(self.df) > 0
        self.features = features
        self.label = label
        self.num_classes = num_classes
        self.y_type = y_type
        self._init_dataset()

    def _init_dataset(self):
        X = torch.tensor(self.df[self.features].values).float()

        y = torch.tensor(self.df[self.label].values)
        if (self.num_classes != len(self.label)) and self.y_type == "bt":
            y = F.one_hot(
                torch.tensor(y).long(), num_classes=self.num_classes
            ).squeeze()

        self.X = X
        self.y = y

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


class DatasetModule(LightningDataModule):
    def __init__(
        self,
        train,
        test,
        batch_size=32,
        features: list = None,
        label: list = None,
        num_classes=2,
        y_type="bt",
        num_workers=4,
    ):
        super().__init__()

        self.batch_size = batch_size
        self.features = features
        self.label = label
        self.num_classes = num_classes
        self.y_type = y_type
        self.num_workers = num_workers

        self._init_dataset(train, test)

    def _init_dataset(self, train, test):
        train, val = train_test_split(train, test_size=0.2)
        print(
            f"Train : {train[self.label].value_counts()}\nval : {val[self.label].value_counts()}\nTest : {test[self.label].value_counts()}"
        )
        if self.y_type == "bt" and len(self.label) == 1:

            class_weights = dict(
                enumerate(
                    class_weight.compute_class_weight(
                        "balanced",
                        classes=np.arange(self.num_classes),
                        y=train[self.label[0]],
                    )
                )
            )
            self.class_weights = class_weights

        self.train = TableDataset(
            train, self.features, self.label, self.num_classes, self.y_type
        )
        self.validation = TableDataset(
            val, self.features, self.label, self.num_classes, self.y_type
        )
        self.test = TableDataset(
            test, self.features, self.label, self.num_classes, self.y_type
        )

    def train_dataloader(self):

        if self.y_type == "bt":
            train_class_weights = [
                self.class_weights[torch.argmax(i).item()] for i in self.train.y
            ]
            sampler = WeightedRandomSampler(
                train_class_weights, len(train_class_weights), replacement=True
            )
        else:
            sampler = RandomSampler(self.train)

        return DataLoader(
            self.train,
            batch_size=self.batch_size,
            sampler=sampler,
            drop_last=True,
            persistent_workers=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            self.validation,
            batch_size=self.batch_size,
            persistent_workers=True,
            num_workers=self.num_workers,
            sampler=SequentialSampler(self.validation),
        )

    def test_dataloader(self):
        return DataLoader(
            self.test,
            batch_size=self.batch_size,
            persistent_workers=True,
            num_workers=self.num_workers,
            sampler=SequentialSampler(self.test),
        )

In [4]:
train_imputed = pd.read_pickle("result/part1/train_imputed.pkl")
test_imputed = pd.read_pickle("result/part1/test_imputed.pkl")

In [5]:
proteomics = test_imputed.columns[test_imputed.columns.tolist().index("C3") :].tolist()
risk_factors = [
    "age",
    "sex",
    "ldl_a",
    "hdl_a",
    "tc_a",
    "tg_a",
    "sbp_a",
    "BMI",
    "smoking",
    "prevalent_diabetes",
]

PRS = ["PRS"]
proteomics

['C3',
 'KLK7',
 'GCHFR',
 'NHLRC3',
 'APOD',
 'GAPDH',
 'TP53I3',
 'CPA4',
 'ANXA2',
 'GRSF1',
 'IL25',
 'HMMR',
 'MRPL52',
 'PAIP2B',
 'THAP12',
 'FOS',
 'FGF9',
 'PITHD1',
 'THSD1',
 'PTGES2',
 'DEFB103A_DEFB103B',
 'ATP1B4',
 'CYB5A',
 'UNC79',
 'SLC34A3',
 'TAGLN3',
 'SLIRP',
 'CLASP1',
 'PSMC3',
 'KIR3DL2',
 'BEX3',
 'PFDN4',
 'BCL7A',
 'SMC3',
 'SLC28A1',
 'CDC123',
 'GJA8',
 'NMRK2',
 'GATA3',
 'CPLX2',
 'RASGRF1',
 'FGF7',
 'ANKRA2',
 'RBM25',
 'LYZL2',
 'CDK1',
 'CREB3',
 'CREBZF',
 'IGLON5',
 'SHC1',
 'ZP4',
 'TMOD4',
 'CEP152',
 'MYH7B',
 'CEP350',
 'CDC25A',
 'TRIM26',
 'MANEAL',
 'MUCL3',
 'GIMAP8',
 'CYTH3',
 'PDXDC1',
 'CLINT1',
 'MAPRE3',
 'EVI2B',
 'STAU1',
 'PCNA',
 'DNAJA1',
 'JMJD1C',
 'GAGE2A',
 'GAD1',
 'IZUMO1',
 'PDCL2',
 'PDE1C',
 'STOML2',
 'BSND',
 'MAPK13',
 'PDIA2',
 'BTLA',
 'MLLT1',
 'TPRKB',
 'ARHGAP5',
 'BTNL10',
 'PHLDB2',
 'PDIA5',
 'ATF4',
 'PRAME',
 'TOP1MT',
 'KHDC3L',
 'DCUN1D2',
 'IL3',
 'DCLRE1C',
 'ERCC1',
 'DCDC2C',
 'VCPKMT',
 'SPRING1',
 'M

In [6]:
dataset = DatasetModule(
    train=train_imputed,
    test=test_imputed,
    features=proteomics,
    label=["incident_cad"],
    num_classes=2,
    batch_size=256,
)

Train : incident_cad
0.0             27209
1.0              1596
dtype: int64
val : incident_cad
0.0             6799
1.0              403
dtype: int64
Test : incident_cad
0.0             14599
1.0               833
dtype: int64


/tmp/ipykernel_23003/1866389822.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y).long(), num_classes=self.num_classes
/tmp/ipykernel_23003/1866389822.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y).long(), num_classes=self.num_classes
/tmp/ipykernel_23003/1866389822.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(y).long(), num_classes=self.num_classes


In [7]:
for x, y in dataset.train_dataloader():
    print(x.shape, y.shape)
    print(torch.argmax(y, dim=1).sum())

torch.Size([256, 2911]) torch.Size([256, 2])
tensor(134)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(130)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(138)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(117)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(130)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(119)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(119)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(138)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(132)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(120)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(131)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(131)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(129)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(127)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(130)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(128)
torch.Size([256, 2911]) torch.Size([256, 2])
tensor(126)
torch.Size([256, 2911]) torch.S

In [8]:
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

import torch
import pytorch_lightning as pl

import torch.nn as nn
import torch.optim as optim
from collections import defaultdict


class LinearResBlock(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearResBlock, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
        self.layer_norm = nn.LayerNorm(output_size)

        torch.nn.init.kaiming_normal_(self.fc1.weight, nonlinearity="relu")  # <6>
        torch.nn.init.constant_(self.layer_norm.weight, 0.5)  # <7>
        torch.nn.init.zeros_(self.layer_norm.bias)

    def forward(self, x):
        out = self.fc1(x)

        out = self.layer_norm(out)
        out = torch.relu(out)
        return out + x


class FullyConnectedNet(pl.LightningModule):
    def __init__(
        self,
        hidden_size,
        features,
        output_size,
        num_resblocks=3,
        lr=1e-3,
        weight_decay=1e-2,
        weight=[1, 1],
        **kwargs,
    ):
        super(FullyConnectedNet, self).__init__()
        input_size = len(features)
        self.features = features
        self.norm = nn.LayerNorm(input_size)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.resblocks = nn.Sequential(
            *[LinearResBlock(hidden_size, hidden_size) for _ in range(num_resblocks)]
        )
        self.fc2 = nn.Linear(hidden_size, output_size)

        self.lr = lr
        self.weight_decay = weight_decay

        self.mertic = {
            "train_auc": torchmetrics.AUROC(num_classes=2, task="multiclass"),
            "val_auc": torchmetrics.AUROC(num_classes=2, task="multiclass"),
        }
        self.history = defaultdict(dict)
        self.loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(weight).float())

    def forward(self, x):
        x = self.norm(x)
        out = torch.relu(self.fc1(x))
        out = self.resblocks(out)
        out = self.fc2(out)
        return out

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        outputs = self.forward(x)
        loss = self.loss_fn(outputs, y.squeeze(-1).float())

        self.mertic["train_auc"].update(
            torch.softmax(outputs, dim=-1), torch.argmax(y, dim=1)
        )

        self.log("ptl/train_loss", loss, on_epoch=True, prog_bar=True, on_step=False)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        outputs = self.forward(x)
        loss = self.loss_fn(outputs, y.squeeze(-1).float())

        self.mertic["val_auc"].update(
            torch.softmax(outputs, dim=-1), torch.argmax(y, dim=1)
        )

        self.log("ptl/val_loss", loss, on_epoch=True, prog_bar=True)

    def on_train_epoch_end(self):

        auc = self.mertic["train_auc"].compute()
        self.log("ptl/train_auc", auc, prog_bar=True)

    def on_validation_epoch_end(self):
        auc = self.mertic["val_auc"].compute()
        self.log("ptl/val_auc", auc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.parameters(), lr=self.lr, weight_decay=self.weight_decay
        )
        return optimizer

    def predict_df(self, df, batch_size=256):

        for feature in self.features:
            assert feature in df.columns
        print(f"input df have NA: {df[self.features].isna().sum(axis=1).sum()}")
        df = df.copy().dropna(subset=self.features)

        predict_dataloader = DataLoader(
            torch.tensor(df[self.features].values).float(),
            batch_size=batch_size,
            persistent_workers=True,
            num_workers=4,
        )

        self.eval()
        pred = []
        with torch.no_grad():
            for x in predict_dataloader:
                y_hat = self.forward(x).cpu().detach()
                y_hat = torch.softmax(y_hat, dim=-1)[:, 1]

                pred.append(y_hat)
        pred = torch.cat(pred).numpy()
        df["pred"] = pred
        return df

In [9]:
used_fatures = proteomics + risk_factors + PRS
used_fatures

['C3',
 'KLK7',
 'GCHFR',
 'NHLRC3',
 'APOD',
 'GAPDH',
 'TP53I3',
 'CPA4',
 'ANXA2',
 'GRSF1',
 'IL25',
 'HMMR',
 'MRPL52',
 'PAIP2B',
 'THAP12',
 'FOS',
 'FGF9',
 'PITHD1',
 'THSD1',
 'PTGES2',
 'DEFB103A_DEFB103B',
 'ATP1B4',
 'CYB5A',
 'UNC79',
 'SLC34A3',
 'TAGLN3',
 'SLIRP',
 'CLASP1',
 'PSMC3',
 'KIR3DL2',
 'BEX3',
 'PFDN4',
 'BCL7A',
 'SMC3',
 'SLC28A1',
 'CDC123',
 'GJA8',
 'NMRK2',
 'GATA3',
 'CPLX2',
 'RASGRF1',
 'FGF7',
 'ANKRA2',
 'RBM25',
 'LYZL2',
 'CDK1',
 'CREB3',
 'CREBZF',
 'IGLON5',
 'SHC1',
 'ZP4',
 'TMOD4',
 'CEP152',
 'MYH7B',
 'CEP350',
 'CDC25A',
 'TRIM26',
 'MANEAL',
 'MUCL3',
 'GIMAP8',
 'CYTH3',
 'PDXDC1',
 'CLINT1',
 'MAPRE3',
 'EVI2B',
 'STAU1',
 'PCNA',
 'DNAJA1',
 'JMJD1C',
 'GAGE2A',
 'GAD1',
 'IZUMO1',
 'PDCL2',
 'PDE1C',
 'STOML2',
 'BSND',
 'MAPK13',
 'PDIA2',
 'BTLA',
 'MLLT1',
 'TPRKB',
 'ARHGAP5',
 'BTNL10',
 'PHLDB2',
 'PDIA5',
 'ATF4',
 'PRAME',
 'TOP1MT',
 'KHDC3L',
 'DCUN1D2',
 'IL3',
 'DCLRE1C',
 'ERCC1',
 'DCDC2C',
 'VCPKMT',
 'SPRING1',
 'M

In [10]:
from ray.train.lightning import (
    RayDDPStrategy,
    RayLightningEnvironment,
    RayTrainReportCallback,
    prepare_trainer,
)
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.train import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.torch import TorchTrainer


def train_func(config):
    dataset = DatasetModule(
        train=train_imputed,
        test=test_imputed,
        features=used_fatures,
        label=["incident_cad"],
        num_classes=2,
        batch_size=config["batch_size"],
    )
    # model = FullyConnectedNet(
    #     input_size=len(proteomics),
    #     hidden_size=config["hidden_size"],
    #     output_size=2,
    #     lr=config["lr"],
    #     weight_decay=config["weight_decay"],
    #     weight=config["weight"],
    #     num_resblocks=config["num_resblocks"],
    # )
    model = FullyConnectedNet(**config)
    trainer = Trainer(
        devices="auto",
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
        enable_progress_bar=True,
    )
    trainer = prepare_trainer(trainer)
    trainer.fit(model, dataset)


search_space = {
    "features": used_fatures,
    "output_size": 2,
    #    "hidden_size": tune.choice([256, 128, 64]),
    "hidden_size": tune.choice([256, 512, 1024]),
    #    "lr": tune.loguniform(1e-4, 1e-1),
    "lr": 0.0074,
    "weight_decay": tune.loguniform(1e-4, 1e-2),
    #    "weight": tune.choice([[1, 1], [0.1, 1], [0.1, 10], [0.1, 100]]),
    "weight": [1, 1],
    "batch_size": 256,
    #    "batch_size": tune.choice([256]),
    "num_resblocks": tune.choice([1, 2, 3, 4, 5]),
}


# The maximum training epochs
num_epochs = 10

# Number of sampls from parameter space
num_samples = 50
scheduler = ASHAScheduler(max_t=num_epochs, grace_period=1, reduction_factor=2)


scaling_config = ScalingConfig(
    num_workers=1, use_gpu=True, resources_per_worker={"CPU": 3, "GPU": 0.5}
)

run_config = RunConfig(
    checkpoint_config=CheckpointConfig(
        num_to_keep=2,
        checkpoint_score_attribute="ptl/val_auc",
        checkpoint_score_order="max",
    ),
)

# Define a TorchTrainer without hyper-parameters for Tuner
ray_trainer = TorchTrainer(
    train_func,
    scaling_config=scaling_config,
    run_config=run_config,
)


def tune_asha(num_samples=10):
    scheduler = ASHAScheduler(max_t=num_epochs, grace_period=1, reduction_factor=2)

    tuner = tune.Tuner(
        ray_trainer,
        param_space={"train_loop_config": search_space},
        tune_config=tune.TuneConfig(
            metric="ptl/val_auc",
            mode="max",
            num_samples=num_samples,
            scheduler=scheduler,
        ),
    )
    return tuner.fit()


results = tune_asha(num_samples=num_samples)


# results.get_best_result("ptl/val_auc")

best_result = results.get_best_result("ptl/val_auc")
best_params = best_result.config
best_result_epoch_dir = (
    best_result.get_best_checkpoint("ptl/val_auc", "max").path + "/checkpoint.ckpt"
)
best_model_state = torch.load(best_result_epoch_dir)
best_model = FullyConnectedNet(**best_params["train_loop_config"])
best_model.load_state_dict(best_model_state["state_dict"])
best_model

(RayTrainWorker pid=16585) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=16464) Started distributed worker processes: 
(TorchTrainer pid=16464) - (ip=172.26.79.196, pid=16585) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=16585) [W Utils.hpp:133] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function getCvarInt)


(RayTrainWorker pid=16585) Train : incident_cad
(RayTrainWorker pid=16585) 0.0             27198
(RayTrainWorker pid=16585) 1.0              1607
(RayTrainWorker pid=16585) dtype: int64
(RayTrainWorker pid=16585) val : incident_cad
(RayTrainWorker pid=16585) 0.0             6810
(RayTrainWorker pid=16585) 1.0              392
(RayTrainWorker pid=16585) dtype: int64
(RayTrainWorker pid=16585) Test : incident_cad
(RayTrainWorker pid=16585) 0.0             14599
(RayTrainWorker pid=16585) 1.0               833
(RayTrainWorker pid=16585) dtype: int64


(RayTrainWorker pid=16593) /tmp/ipykernel_23003/1866389822.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
(RayTrainWorker pid=16593) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=16585) GPU available: True (cuda), used: True
(RayTrainWorker pid=16585) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=16585) IPU available: False, using: 0 IPUs
(RayTrainWorker pid=16585) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=16585) /home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
(RayTrainWorker pid=16585) You are using a CUDA device ('NVIDIA GeForce RTX 4060') that has Tensor Cores. To properly utilize them, you shoul

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:03<00:00,  0.54it/s]
(RayTrainWorker pid=16593) Train : incident_cad
(RayTrainWorker pid=16593) 1.0               833 [repeated 6x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RayTrainWorker pid=16593) dtype: int64 [repeated 3x across cluster]
(RayTrainWorker pid=16593) val : incident_cad
(RayTrainWorker pid=16593) Test : incident_cad


(RayTrainWorker pid=16585) /home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:441: It is recommended to use `self.log('ptl/val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
(RayTrainWorker pid=16585) /home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:441: It is recommended to use `self.log('ptl/val_auc', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
(RayTrainWorker pid=16585) /tmp/ipykernel_23003/1866389822.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor). [repeated 5x across cluster]
(RayTrainWorker pid=16593) GPU availab

...

(RayTrainWorker pid=22727) /home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:441: It is recommended to use `self.log('ptl/val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
(RayTrainWorker pid=22727) /home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:441: It is recommended to use `self.log('ptl/val_auc', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.


Epoch 1:   7%|▋         | 8/111 [00:00<00:01, 71.09it/s, v_num=0, ptl/val_loss=0.602, ptl/val_auc=0.752, ptl/train_loss=0.588, ptl/train_auc=0.792]


(RayTrainWorker pid=22727) /home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:441: It is recommended to use `self.log('ptl/train_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
(RayTrainWorker pid=22727) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/xutingfeng/ray_results/TorchTrainer_2024-04-17_10-26-51/TorchTrainer_f5695_00023_23_hidden_size=256,num_resblocks=2,weight_decay=0.0005_2024-04-17_10-26-56/checkpoint_000000)
(RayTrainWorker pid=22727) /home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:441: It is recommended to use `self.log('ptl/train_auc', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.


Epoch 1:  69%|██████▉   | 77/111 [00:00<00:00, 82.72it/s, v_num=0, ptl/val_loss=0.602, ptl/val_auc=0.752, ptl/train_loss=0.588, ptl/train_auc=0.792]
(RayTrainWorker pid=22727)  [repeated 5x across cluster]
Epoch 1:  86%|████████▋ | 96/111 [00:01<00:00, 83.37it/s, v_num=0, ptl/val_loss=0.602, ptl/val_auc=0.752, ptl/train_loss=0.588, ptl/train_auc=0.792] [repeated 4x across cluster]


(RayTrainWorker pid=22727) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/xutingfeng/ray_results/TorchTrainer_2024-04-17_10-26-51/TorchTrainer_f5695_00023_23_hidden_size=256,num_resblocks=2,weight_decay=0.0005_2024-04-17_10-26-56/checkpoint_000001)
2024-04-17 10:31:10,511	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).


In [38]:
results.get_dataframe().sort_values("ptl/val_auc")

,ptl/val_loss,ptl/val_auc,ptl/train_loss,epoch,step,timestamp,checkpoint_dir_name,should_checkpoint,done,training_iteration,...,config/train_loop_config/features,config/train_loop_config/output_size,config/train_loop_config/hidden_size,config/train_loop_config/lr,config/train_loop_config/weight_decay,config/train_loop_config/weight,config/train_loop_config/batch_size,config/train_loop_config/num_resblocks,ptl/train_auc,logdir
18,1.760173,0.469385,1.876581,0,111,1713250225,checkpoint_000000,True,True,1,...,"[C3, KLK7, GCHFR, NHLRC3, APOD, GAPDH, TP53I3,...",2,128,0.039256,0.000113,"[0.1, 10]",256,5,NaN,b1adb_00018
2,14.133671,0.483149,4.743559,0,111,1713250064,checkpoint_000000,True,True,1,...,"[C3, KLK7, GCHFR, NHLRC3, APOD, GAPDH, TP53I3,...",2,256,0.019132,0.004371,"[0.1, 100]",256,5,NaN,b1adb_00002
27,130.268478,0.493366,50.511715,0,111,1713250306,checkpoint_000000,True,True,1,...,"[C3, KLK7, GCHFR, NHLRC3, APOD, GAPDH, TP53I3,...",2,128,0.056077,0.008174,"[0.1, 100]",256,3,NaN,b1adb_00027
46,0.742477,0.494961,1.494878,0,111,1713250502,checkpoint_000000,True,True,1,...,"[C3, KLK7, GCHFR, NHLRC3, APOD, GAPDH, TP53I3,...",2,256,0.060112,0.002438,"[1, 1]",256,3,NaN,b1adb_00046
38,19.235968,0.497104,19.780983,0,111,1713250422,checkpoint_000000,True,True,1,...,"[C3, KLK7, GCHFR, NHLRC3, APOD, GAPDH, TP53I3,...",2,256,0.073539,0.001687,"[0.1, 10]",256,4,NaN,b1adb_00038
40,2.810828,0.502995,22.124250,0,111,1713250441,checkpoint_000000,True,True,1,...,"[C3, KLK7, GCHFR, NHLRC3, APOD, GAPDH, TP53I3,...",2,128,0.046260,0.000291,"[0.1, 100]",256,4,NaN,b1adb_00040
37,0.408884,0.506463,1.211488,0,111,1713250415,checkpoint_000000,True,True,1,...,"[C3, KLK7, GCHFR, NHLRC3, APOD, GAPDH, TP53I3,...",2,128,0.029619,0.003661,"[0.1, 10]",256,5,NaN,b1adb_00037
26,0.747489,0.515900,1.141289,0,111,1713250301,checkpoint_000000,True,True,1,...,"[C3, KLK7, GCHFR, NHLRC3, APOD, GAPDH, TP53I3,...",2,64,0.079681,0.000378,"[1, 1]",256,4,NaN,b1adb_00026
23,0.985663,0.528829,5.112461,0,111,1713250269,checkpoint_000000,True,True,1,...,"[C3, KLK7, GCHFR, NHLRC3, APOD, GAPDH, TP53I3,...",2,64,0.082075,0.004236,"[0.1, 100]",256,4,NaN,b1adb_00023
22,9.694574,0.533914,9.452261,0,111,1713250263,checkpoint_000000,True,True,1,...,"[C3, KLK7, GCHFR, NHLRC3, APOD, GAPDH, TP53I3,...",2,64,0.061577,0.007046,"[0.1, 10]",256,3,NaN,b1adb_00022


In [39]:
# results.get_best_result("ptl/val_auc")

best_result = results.get_best_result("ptl/val_auc")
best_params = best_result.config
best_result_epoch_dir = (
    best_result.get_best_checkpoint("ptl/val_auc", "max").path + "/checkpoint.ckpt"
)
best_model_state = torch.load(best_result_epoch_dir)
best_model = FullyConnectedNet(**best_params["train_loop_config"])
best_model.load_state_dict(best_model_state["state_dict"])
best_model

FullyConnectedNet(
  (norm): LayerNorm((2922,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=2922, out_features=256, bias=True)
  (resblocks): Sequential(
    (0): LinearResBlock(
      (fc1): Linear(in_features=256, out_features=256, bias=True)
      (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (1): LinearResBlock(
      (fc1): Linear(in_features=256, out_features=256, bias=True)
      (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (2): LinearResBlock(
      (fc1): Linear(in_features=256, out_features=256, bias=True)
      (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
  )
  (fc2): Linear(in_features=256, out_features=2, bias=True)
  (loss_fn): CrossEntropyLoss()
)

In [40]:
best_params["train_loop_config"]

{'features': ['C3',
  'KLK7',
  'GCHFR',
  'NHLRC3',
  'APOD',
  'GAPDH',
  'TP53I3',
  'CPA4',
  'ANXA2',
  'GRSF1',
  'IL25',
  'HMMR',
  'MRPL52',
  'PAIP2B',
  'THAP12',
  'FOS',
  'FGF9',
  'PITHD1',
  'THSD1',
  'PTGES2',
  'DEFB103A_DEFB103B',
  'ATP1B4',
  'CYB5A',
  'UNC79',
  'SLC34A3',
  'TAGLN3',
  'SLIRP',
  'CLASP1',
  'PSMC3',
  'KIR3DL2',
  'BEX3',
  'PFDN4',
  'BCL7A',
  'SMC3',
  'SLC28A1',
  'CDC123',
  'GJA8',
  'NMRK2',
  'GATA3',
  'CPLX2',
  'RASGRF1',
  'FGF7',
  'ANKRA2',
  'RBM25',
  'LYZL2',
  'CDK1',
  'CREB3',
  'CREBZF',
  'IGLON5',
  'SHC1',
  'ZP4',
  'TMOD4',
  'CEP152',
  'MYH7B',
  'CEP350',
  'CDC25A',
  'TRIM26',
  'MANEAL',
  'MUCL3',
  'GIMAP8',
  'CYTH3',
  'PDXDC1',
  'CLINT1',
  'MAPRE3',
  'EVI2B',
  'STAU1',
  'PCNA',
  'DNAJA1',
  'JMJD1C',
  'GAGE2A',
  'GAD1',
  'IZUMO1',
  'PDCL2',
  'PDE1C',
  'STOML2',
  'BSND',
  'MAPK13',
  'PDIA2',
  'BTLA',
  'MLLT1',
  'TPRKB',
  'ARHGAP5',
  'BTNL10',
  'PHLDB2',
  'PDIA5',
  'ATF4',
  'PRAME',
  

In [41]:
test_imputed = best_model.predict_df(test_imputed)
test_imputed

input df have NA: 0


,eid,PRS,sex,height,weight,BSA,genotype_array,age,PC1,PC2,...,TGFBR3,CRTAC1,IGFBP7,SELE,VWF,NOTCH3,CNTN1,ENG,ICAM2,pred
19409,2883530.0,1.030583,1.0,171.0,64.2,1.746282,2,44.0,71.3002,-100.66700,...,-0.0087,-0.029539,0.022568,-0.027118,0.008048,0.004249,0.000619,0.001707,-0.026825,0.307764
19272,2867444.0,2.192278,0.0,165.0,55.8,1.599219,2,53.0,-12.4815,3.16181,...,0.1859,0.291950,0.147400,-0.120500,0.597300,0.115700,0.243300,0.127800,0.063400,0.553233
49865,5869793.0,0.653794,1.0,171.0,77.3,1.916181,2,62.0,-11.4721,2.20519,...,0.0516,0.369750,-0.155300,0.035500,-0.276700,-0.043900,0.195500,-0.111000,-0.990800,0.469619
39664,4880838.0,0.664819,0.0,163.0,84.4,1.954852,2,62.0,-11.1640,3.66252,...,-0.0127,0.393200,0.174500,0.035700,0.873200,0.236600,0.114200,0.134400,0.008700,0.095878
30555,3987428.0,0.826465,0.0,164.0,73.1,1.824859,1,66.0,-11.4666,2.77498,...,-0.5216,0.005050,-0.160200,0.181900,1.026700,-0.062150,-0.094500,-0.032700,0.213200,0.454483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43327,5241912.0,1.085083,1.0,176.0,116.0,2.381409,2,45.0,-10.8083,4.46241,...,0.2348,-0.919950,0.803300,0.131600,0.481500,0.279800,-0.226200,0.262600,0.239400,0.859506
29129,3851862.0,1.294348,0.0,169.0,72.9,1.849932,2,40.0,-12.6549,3.40064,...,-0.3290,-0.251250,-0.787400,-0.919000,0.212700,-0.617800,0.123900,-0.124100,-0.940500,0.082682
1550,1144512.0,0.722791,1.0,191.0,96.6,2.263883,1,59.0,-12.7237,1.46547,...,0.1043,-0.284750,0.350300,1.608600,-0.341300,0.134100,-0.012000,0.226700,0.135200,0.406253
1888,1177099.0,1.335307,1.0,175.0,75.1,1.910679,2,63.0,-15.1573,7.36690,...,0.2172,0.172250,0.431300,0.121750,-0.754900,0.530700,0.244000,-0.018900,-0.053400,0.468948


In [1]:
cal_binary_metrics(test_imputed["incident_cad"], test_imputed["pred"])

NameError: name 'cal_binary_metrics' is not defined

In [17]:
import pandas as pd
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    f1_score,
    roc_curve,
    precision_recall_curve,
    auc,
)


from tqdm.rich import tqdm
import numpy as np

from statsmodels.stats.multitest import multipletests


import statsmodels.api as sm

from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
)
from scipy.stats import pearsonr, spearmanr


def generate_multipletests_result(df, pvalue_col="pvalue", alpha=0.05, method="fdr_bh"):
    df = df.copy()
    pvalue_series = df[pvalue_col]
    reject, pvals_corrected, _, _ = multipletests(
        pvalue_series, alpha=alpha, method="fdr_bh"
    )
    df["pval_corrected"] = pvals_corrected
    df["reject"] = reject
    return df


def find_best_cutoff(fpr, tpr, thresholds):
    diff = tpr - fpr
    Youden_index = np.argmax(diff)
    optimal_threshold = thresholds[Youden_index]
    optimal_FPR, optimal_TPR = fpr[Youden_index], tpr[Youden_index]
    return optimal_threshold, optimal_FPR, optimal_TPR


def cal_binary_metrics(y, y_pred):
    fpr, tpr, thresholds = roc_curve(y, y_pred)
    AUC = roc_auc_score(y, y_pred)
    # by best youden

    optim_threshold, optim_fpr, optim_tpr = find_best_cutoff(fpr, tpr, thresholds)
    y_pred_binary = (y_pred > optim_threshold).astype(int)
    ACC = accuracy_score(y, y_pred_binary)
    macro_f1 = f1_score(y, y_pred_binary, average="macro")
    sensitivity = optim_tpr
    specificity = 1 - optim_fpr
    precision, recall, _ = precision_recall_curve(y, y_pred)
    APR = auc(recall, precision)

    return {
        "AUC": AUC,
        "ACC": ACC,
        "Macro_F1": macro_f1,
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "APR": APR,
    }


import torch.nn as nn
import torch.optim as optim
from tqdm.rich import tqdm

# 定义神经网络模型


class LinearResBlock(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearResBlock, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
        self.batch_norm = nn.LayerNorm(output_size)

        torch.nn.init.kaiming_normal_(self.fc1.weight, nonlinearity="relu")  # <6>

        torch.nn.init.constant_(self.batch_norm.weight, 0.5)  # <7>
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.fc1(x)

        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x


class FullyConnectedNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_resblocks=3):
        super(FullyConnectedNet, self).__init__()
        self.norm = nn.LayerNorm(input_size)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.resblocks = nn.Sequential(
            *[LinearResBlock(hidden_size, hidden_size) for _ in range(num_resblocks)]
        )
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.norm(x)
        out = torch.relu(self.fc1(x))
        out = self.resblocks(out)
        out = self.fc2(out)
        return out


# 定义训练函数
def train(model, dataset, criterion, optimizer, num_epochs):
    train_loader = dataset.train_dataloader()
    val_loader = dataset.test_dataloader()
    for epoch in range(num_epochs):
        running_loss = 0.0
        auroc = torchmetrics.AUROC(num_classes=2, task="multiclass")
        for inputs, labels in tqdm(
            train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", total=len(train_loader)
        ):
            inputs, labels = inputs.to("cuda:0"), labels.to("cuda:0")
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.squeeze(-1).float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            auroc.update(torch.softmax(outputs, dim=-1), torch.argmax(labels, dim=1))
        auc = auroc.compute()
        print(
            f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, AUC: {auc}"
        )
        if epoch % 1 == 0:
            test_auc = torchmetrics.AUROC(num_classes=2, task="multiclass")
            for inputs, labels in val_loader:
                inputs, labels = inputs.to("cuda:0"), labels.to("cuda:0")
                outputs = model(inputs)
                test_auc.update(
                    torch.softmax(outputs, dim=-1), torch.argmax(labels, dim=1)
                )
            print(f"Epoch {epoch+1}/{num_epochs}, Test AUC: {test_auc.compute()}")
    # test_auc = torchmetrics.AUROC(num_classes=2, task="multiclass")
    # for inputs, labels in dataset.test_dataloader():
    #     inputs, labels = inputs.to("cuda:0"), labels.to("cuda:0")
    #     outputs = model(inputs)
    #     test_auc.update(torch.softmax(outputs, dim=-1), torch.argmax(labels, dim=1))
    # print(f"Test AUC: {test_auc.compute()}")

In [ ]:
from pytorch_lightning import Trainer, seed_everything

seed_everything(42)

In [ ]:
# 设置训练参数
input_size = len(proteomics)  # 输入特征维度
hidden_size = 512  # 隐藏层维度
output_size = 2  # 输出类别数
learning_rate = 5e-4
batch_size = 256
num_epochs = 5


# 创建模型实例
best_model = FullyConnectedNet(
    input_size=len(proteomics), hidden_size=256, output_size=2, num_resblocks=6
)
best_model.to("cuda:0")
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss(weight=torch.Tensor([0.1, 100]).to("cuda:0"))
optimizer = optim.NAdam(best_model.parameters(), lr=learning_rate, weight_decay=5e-3)


# 开始训练
train(best_model, dataset, criterion, optimizer, num_epochs)

In [ ]:
y_pred_list = []
y_list = []
AUC = torchmetrics.AUROC(num_classes=2, task="multiclass")

best_model.eval()
for x, y in dataset.test_dataloader():
    y_pred = best_model(x.to("cuda:0")).cpu().detach()
    y_pred_list.append(y_pred)
    y_list.append(y)
    AUC.update(torch.softmax(y_pred, dim=-1), torch.argmax(y, dim=1))

AUC_values = AUC.compute()
print(f"AUC: {AUC_values}")

In [ ]:
y_pred = torch.softmax(torch.cat(y_pred_list), dim=-1)[:, 1].numpy()
y_true = torch.argmax(torch.cat(y_list), dim=1).numpy()

test_df = pd.DataFrame(
    {
        "y_pred": y_pred,
        "y_true": y_true,
    }
)

cal_binary_metrics(y_true, y_pred)

In [ ]:
y_true

In [ ]:
cal_binary_metrics(test_df["y_true"], test_df["y_pred"])